In [64]:
import numpy as np
import pandas as pd
import copy
import gc
import time
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.decomposition import PCA
import geopandas as gpd

import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge,Lasso
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN,KMeans
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
import catboost
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb
from xgboost import plot_importance, plot_tree
import lightgbm as lgb

import torch 
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
from torch.nn import functional as F 
from scipy.spatial.distance import pdist, squareform
%matplotlib inline
from joblib import dump, load
import geopandas as gpd
from sklearn.model_selection import KFold
from shapely.geometry import Polygon,Point


kf=KFold(n_splits=5,shuffle=True,random_state=2024)

In [66]:
#############################################
#########  想看数据路径就用下面的代码   #########
#############################################

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        

# 读取数据

+ train = 训练集
+ test = 测试集
+ train_addition = 天气数据
+ test_ additon = 天气数据
+ train_aug = osrm 道路数据
+ test_aug = osrm 道路数据

+ ori_train = 训练数据副本
+ test_train = 测试数据副本

不用管，可以拿最终的train和test训练，可以一直执行到下一段MarkDown文本

In [67]:
train=pd.read_csv("/kaggle/input/nyc-taxi-trip-duration/train.zip")
test=pd.read_csv("/kaggle/input/nyc-taxi-trip-duration/test.zip")

In [68]:
train_aug=pd.read_csv('/kaggle/input/nyc-taxi-trip-noisy/train_augmented.csv')
test_aug=pd.read_csv('/kaggle/input/nyc-taxi-trip-noisy/test_augmented.csv')

In [69]:
train=train.merge(train_aug,on='id',how='left')
test=test.merge(test_aug,on='id',how='left')

In [70]:
train=pd.concat([train,pd.read_csv("/kaggle/input/nyc-addtion/Addition.csv")],axis=1)
test=pd.concat([test,pd.read_csv("/kaggle/input/nyc-addtion/New York City Taxi Trip Duration/Addition_test.csv")],axis=1)

In [71]:
ori_train,ori_test=copy.deepcopy(train),copy.deepcopy(test)

In [72]:
weather = pd.read_csv("/kaggle/input/weather-data-in-new-york-city-2016/weather_data_nyc_centralpark_2016(1).csv")
weather['precipitation'] = pd.to_numeric(weather['precipitation'], errors='coerce')
weather['snow fall'] = pd.to_numeric(weather['snow fall'], errors='coerce')
weather['snow depth'] = pd.to_numeric(weather['snow depth'], errors='coerce')

# 下面是聚类点数据，可以不使用

In [73]:
test_C=load("/kaggle/input/nyc-addtion/cluster (2).pkl")
train['in_Cluster']=test_C.predict(train[['pickup_latitude','pickup_longitude']].values)
train['out_Cluster']=test_C.predict(train[['dropoff_latitude','dropoff_longitude']].values)
test['in_Cluster']=test_C.predict(test[['pickup_latitude','pickup_longitude']].values)
test['out_Cluster']=test_C.predict(test[['dropoff_latitude','dropoff_longitude']].values)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 1.0.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# 数据清洗

+ 把自己的步骤导入进来

In [97]:
def limit_range(train):
    train = train[train['pickup_longitude'].between(-75, -73)]
    train = train[train['pickup_latitude'].between(40, 42)]
    train = train[train['dropoff_longitude'].between(-75, -73)]
    train = train[train['dropoff_latitude'].between(40, 42)]
    return train

In [98]:
train,test=limit_range(train),limit_range(test)

In [99]:
# 所有工作做完之后
test_index=test.index # 清洗后的索引
train_index=train.index

In [100]:
train.reset_index(inplace=True) # 做也可以不做也可以
test.reset_index(inplace=True)

# 特征工程

+ 这里面替换成自己的就行
+ 或者有些一些想要保留的就直接保留了
+ 添加自己的特征列，最后在features里面输入就行

+ 对于train的处理： train=process(train)
+ 对于test的处理： test=process(test,False)

In [74]:
def process(train,is_Train=True,is_outrange=False):

    ########################################################################################################
    ############                      时间特征                               ###############################
    ########################################################################################################

    def rush_hour_f(row):
        rhour = row['real_hour']
        if (6 <= rhour) & (rhour <= 10):
            return 1
        if (10 < rhour) & (rhour < 16):
            return 2
        if (16 <= rhour) & (rhour <= 20):
            return 3
        return 0
    def people(data):
        if data==0:
            return 1
        if data>6:return 6
        return data
    train['pc']=train['passenger_count'].apply(people)
    


    encoder.fit(train['store_and_fwd_flag'])
    train['store_and_fwd_flag'] = encoder.transform(train['store_and_fwd_flag'])
    
    train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'])
    train['month'] = train['pickup_datetime'].dt.month
    train['weekday'] = train['pickup_datetime'].dt.weekday
    train['hour'] = train['pickup_datetime'].dt.hour
    train['minute'] = train['pickup_datetime'].dt.minute
    train['minute_of_day'] = train['hour'] * 60 + train['minute']
    train['real_hour'] = train['minute_of_day'] / 60
        #for weather
    train['year'] = train['pickup_datetime'].dt.year
    train['day'] = train['pickup_datetime'].dt.day
    train['rush_hour'] = train.apply(rush_hour_f, axis=1)

    cal = calendar()
    holidays = cal.holidays(start=pd.Timestamp(2015,12,31), end=pd.Timestamp(2017,1,1))
    H=[[i.month,i.day] for i in holidays]
    train['is_weekend']=train['weekday'].apply(lambda x:1 if x>4 else 0)
    for i,j in H:
        # 这里特别值得注意，查询返回的是一个视图
        # 对视图的修改并不会返回到原始上
        # 因此，我们需要做的是，采用iloc获得条件查询的index并进行修改
        train.iloc[train[(train['month']==i)&(train['day']==j)].index,-1]=1
    encoder.fit(train['is_weekend'])
    train['is_weekend'] = encoder.transform(train['is_weekend'])
    
    ########################################################################################################
    ############                      空间特征                               ###############################
    ########################################################################################################
    AVG_EARTH_RADIUS =6369
    def ft_haversine_distance(lat1, lng1, lat2, lng2):
    # Haversine距离，用于测量大地距离
        lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
        lat = lat2 - lat1
        lng = lng2 - lng1
        d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
        h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
        return h

    def ft_degree(lat1,log1,lat2,log2):
        # 用于计算坐标方位
        lng_delta_rad = np.radians(log2 - log1)
        lat1, lng1, lat2, lng2 = map(np.radians, (lat1,log1,lat2,log2))
        y = np.sin(lng_delta_rad) * np.cos(lat2)
        x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
        return np.degrees(np.arctan2(y, x))     
    def real_km(lat1,log1,lat2,log2):
        theta=np.cos(np.mean(lat1+lat2))
        return np.abs(lat1-lat2)*111.1+np.abs(log1-log2)*111.1*theta
    def direction_bins(data,k=16):
        node=360//k
        data+=180
        for i in range(k):
            if node*i<=data<node*(i+1):
                return i
        return i
    def Type(data):
        if data<=0.1:
            return 0
        if data<=1:
            return 1
        if data<=3:
            return 2
        return 3
        
    train['distance_have']=ft_haversine_distance(train['pickup_latitude'].values,
                                                     train['pickup_longitude'].values, 
                                                     train['dropoff_latitude'].values,
                                                     train['dropoff_longitude'].values)
    train['distance']=train['distance']/1000
    train['distance_type']=train['distance'].apply(Type)

    train['distance_log']=np.log(train.distance+1)
    train['duration_log']=np.log(train.duration+1)
    train['direction'] = ft_degree(train['pickup_latitude'].values,
                                    train['pickup_longitude'].values,
                                    train['dropoff_latitude'].values,
                                    train['dropoff_longitude'].values)

    train['direction_bins']=train['direction'].apply(direction_bins)
    train['real_km']=real_km(train['pickup_latitude'].values,
                                    train['pickup_longitude'].values,
                                    train['dropoff_latitude'].values,
                                    train['dropoff_longitude'].values)
    train['ave_speed']=train.distance/(train.duration+1e-3)*3600
    train['day_of_year']=train['day']+(train['month']-1)*31
    if is_Train:
        train = train[(train.trip_duration < 1000000)] if not is_outrange else train
        duration = train['trip_duration']
        train['speed'] = train.distance_have / duration * 2236.936292
        train['trip_duration'] = np.log(train['trip_duration'].values)
        
    ########################################################################################################
    ############                      附带信息特征                               ###############################
    ########################################################################################################
        #NYS max speed limit 55mph

    weather["date"] = pd.to_datetime(weather["date"],format='%d-%m-%Y')
    weather['year'] = weather['date'].dt.year
    weather_2016 = weather[weather["year"] == 2016]
    weather_2016.drop(["year"], axis=1, inplace=True)
    
    train['date']=pd.to_datetime(train[['year','month','day']],errors='coerce')
    left_merge = pd.merge(left=train, right=weather_2016, on="date", how="left")
    train = left_merge.loc[:, left_merge.columns != 'date']
    
    if not is_outrange:
        train['in_Cluster']=train['in_Cluster'].astype(str)
        train['out_Cluster']=train['out_Cluster'].astype(str)

        encoder.fit(train['in_Cluster'])
        train['in_Cluster']=encoder.transform(train['in_Cluster'])
        encoder.fit(train['out_Cluster'])
        train['out_Cluster']=encoder.transform(train['out_Cluster'])
    
    return train
        

In [75]:
train,test=process(train),process(test,False)

/tmp/ipykernel_36/2917683.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['speed'] = train.distance_have / duration * 2236.936292
/tmp/ipykernel_36/2917683.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['trip_duration'] = np.log(train['trip_duration'].values)
/tmp/ipykernel_36/2917683.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

# 特征选择

+ 往features添加特征列名称就行

In [76]:
columns=[ 'vendor_id','distance','duration_log', 'distance_have',
         'motorway', 'trunk', 'primary',
       'secondary', 'tertiary', 'unclassified', 'residential',
       'nTrafficSignals', 'nCrossing', 'nStop', 'nIntersection',
       'pc', 'pickup_longitude', 'pickup_latitude','day_of_year',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'month', 'weekday', 
          'hour', 'minute',
       'real_hour', 'rush_hour', 'is_weekend', 'year', 'day', 
          'humidity', 'pressure', 'temperature',
        'wind_speed',
       'direction_bins', 'maximum temperature', 'minimum temperature',
       'average temperature', 'precipitation', 'snow fall',
         'snow depth',
         'in_Cluster','out_Cluster',
         
         'speed','trip_duration'
         ]
features=columns[:-2]

# 模型训练

+ 最终提交请使用get_oof这里的交叉验证模型
+ API 使用也很简单，参考： 

```python
%%time
x=train[features]
y=train.trip_duration
pred_train3,pred_test3=get_oof_ligGBM(x,y,test[features])
```

In [91]:
def get_oof_ligGBM(x_train,y_train,x_test,p=None):
    
    lgb_params = {
'learning_rate': 0.1,
'max_depth': 16,
'num_leaves':1000, 
'objective': 'regression',
'feature_fraction': 0.8,
'bagging_fraction': 0.6,
'max_bin': 1000
,'verbosity': -1,
    'metric':'rmse',
        'verbose_eval':10
} if p==None else p
    
    oof_train=np.zeros((x_train.shape[0],))
    oof_test=np.zeros((x_test.shape[0],))


    for i,(train_index,valid_index) in enumerate(kf.split(x_train,y_train)):
        trn_x,trn_y=x_train.iloc[train_index],y_train.iloc[train_index]
        val_x,val_y=x_train.iloc[valid_index],y_train.iloc[valid_index]
        dtrn=lgb.Dataset(trn_x,trn_y)
        dval=lgb.Dataset(val_x,val_y)
        bst=lgb.train(lgb_params, dtrn, num_boost_round=1500,valid_sets=[dtrn,dval])
        
        oof_train[valid_index]=bst.predict(val_x)
        oof_test+=bst.predict(x_test)/5
    return oof_train.reshape(-1,1),oof_test.reshape(-1,1)

In [92]:
def get_oof_XGBoost(x_train,y_train,x_test,p=None):
    # 算法参数
    params = {
        # 通用参数
    'min_child_weight': 50,
    'learning_rate': 0.05,
    'colsample_bytree': 0.7,
    'max_depth': 16,
    'subsample': 0.8,
    'booster' : 'gbtree',
    'eval_metric': 'rmse',
    'objective': 'reg:squarederror',
    'colsample_bylevel': 0.7,
    'silent':1,
    'reg_alpha':1,
        # 'eval_metric': 'auc'
    } if p==None else p
    
    oof_train=np.zeros((x_train.shape[0],))
    oof_test=np.zeros((x_test.shape[0],))


    for i,(train_index,valid_index) in enumerate(kf.split(x_train,y_train)):
        trn_x,trn_y=x_train.iloc[train_index],y_train.iloc[train_index]
        val_x,val_y=x_train.iloc[valid_index],y_train.iloc[valid_index]
        dtrn=xgb.DMatrix(trn_x,trn_y)
        dval=xgb.DMatrix(val_x,val_y)
        watchlist=[(dtrn,"train"),(dval,"valid_data")]
        bst=xgb.train(params, dtrn, num_boost_round=7,evals=watchlist,early_stopping_rounds=50,verbose_eval=100)
        oof_test+=bst.predict(xgb.DMatrix(x_test))/5
        oof_train[valid_index]=bst.predict(xgb.DMatrix(val_x))

    return oof_train.reshape(-1,1),oof_test.reshape(-1,1)

In [79]:
def get_oof_CatBoost(x_train,y_train,x_test,p=None):
    # 算法参数
    params={
    'loss_function': 'RMSE', # 损失函数，取值RMSE, Logloss, MAE, CrossEntropy, Quantile, LogLinQuantile, Multiclass, MultiClassOneVsAll, MAPE, Poisson。默认Logloss。

    'iterations': 700, # 最大迭代次数，默认500. 别名：num_boost_round, n_estimators, num_trees
    'learning_rate': 0.1, # 学习速率,默认0.03 别名：eta
    'random_seed': 2024, # 训练的随机种子，别名：random_state
    'l2_leaf_reg': 1, # l2正则项，别名：reg_lambda
    'bootstrap_type': 'Bernoulli', # 确定抽样时的样本权重，取值Bayesian、Bernoulli(伯努利实验)、MVS(仅支持cpu)、Poisson(仅支持gpu)、No（取值为No时，每棵树为简单随机抽样）;默认值GPU下为Bayesian、CPU下为MVS
#     'bagging_temperature': 0,  # bootstrap_type=Bayesian时使用,取值为1时采样权重服从指数分布；取值为0时所有采样权重均等于1。取值范围[0，inf)，值越大、bagging就越激进
    'subsample': 0.6, # 样本采样比率（行采样）
    'sampling_frequency': 'PerTree', # 采样频率，取值PerTree（在构建每棵新树之前采样）、PerTreeLevel（默认值，在子树的每次分裂之前采样）；仅支持CPU
    'use_best_model': True, # 让模型使用效果最优的子树棵树/迭代次数，使用验证集的最优效果对应的迭代次数（eval_metric：评估指标，eval_set：验证集数据），布尔类型可取值0，1（取1时要求设置验证集数据）
    'best_model_min_trees': 50, # 最少子树棵树,和use_best_model一起使用
    'depth': 10, # 树深，默认值6
    'grow_policy': 'SymmetricTree', # 子树生长策略，取值SymmetricTree（默认值，对称树）、Depthwise（整层生长，同xgb）、Lossguide（叶子结点生长，同lgb）
    'min_data_in_leaf': 500, # 叶子结点最小样本量
#     'max_leaves': 12, # 最大叶子结点数量
    'one_hot_max_size': 4, # 对唯一值数量<one_hot_max_size的类别型特征使用one-hot编码
    'rsm': 0.6, # 列采样比率，别名colsample_bylevel 取值（0，1],默认值1
    'nan_mode': 'Max', # 缺失值处理方法，取值Forbidden（不支持缺失值，输入包含缺失时会报错）、Min（处理为该列的最小值，比原最小值更小）、Max（同理）
    'input_borders': None, # 特征数据边界（最大最小边界）、会影响缺失值的处理（nan_mode取值Min、Max时），默认值None、在训练时特征取值的最大最小值即为特征值边界
    'boosting_type': 'Ordered', # 提升类型，取值Ordered（catboost特有的排序提升，在小数据集上效果可能更好，但是运行速度较慢）、Plain（经典提升）
    'max_ctr_complexity':4, # 分类特征交叉的最高阶数，默认值4
#     'logging_level':'Silent', # 模型训练过程的信息输出等级，取值Silent（不输出信息）、Verbose（默认值，输出评估指标、已训练时间、剩余时间等）、Info（输出额外信息、树的棵树）、Debug（debug信息）
    'metric_period': 100, # 计算目标值、评估指标的频率，默认值1、即每次迭代都输出目标值、评估指标
    'early_stopping_rounds': 50,
    'border_count': 300, # 数值型特征的分箱数，别名max_bin，取值范围[1,65535]、默认值254（CPU下), # 设置提前停止训练，在得到最佳的评估结果后、再迭代n（参数值为n）次停止训练，默认值不启用
    'feature_border_type': 'GreedyLogSum', # 数值型特征的分箱方法，取值Median、Uniform、UniformAndQuantiles、MaxLogSum、MinEntropy、GreedyLogSum（默认值）
        'od_type': 'Iter',

        
} if p==None else p
    
    oof_train=np.zeros((x_train.shape[0],))
    oof_test=np.zeros((x_test.shape[0],))


    for i,(train_index,valid_index) in enumerate(kf.split(x_train,y_train)):
        bst=CatBoostRegressor(**params)
        trn_x,trn_y=x_train.iloc[train_index],y_train.iloc[train_index]
        val_x,val_y=x_train.iloc[valid_index],y_train.iloc[valid_index]
       
        bst.fit(trn_x,trn_y,eval_set=(val_x,val_y))
        oof_test+=bst.predict(x_test)/5
        oof_train[valid_index]=bst.predict(val_x)

    return oof_train.reshape(-1,1),oof_test.reshape(-1,1)


#  结果提交
+ queryScore是查询分数的
+ makeRes是直接输出结果的

用法：

```python
# queryScore
%%time
queryScore(你的模型，训练集，测试集)
```

```python
# makeRes
makeRes(预测结果，文件名)

makeRes(np.exp(pred3),'xgboost_1')

```

In [60]:
def queryScore(model,train=None,test=None):
    print("*"*10)
    print("Train shape",trainXsplit[features].shape)
    print("Score in Train %.10f"%np.sqrt(MSE(model.predict(trainXsplit[features]),trainXsplit.trip_duration)))
    print("Val shape",testXsplit[features].shape)
    print("*"*10)
    print("Score in Val %.10f"%np.sqrt(MSE(model.predict(testXsplit[features]),testXsplit.trip_duration)))

In [61]:
def makeRes(pred,p):
    submission = pd.DataFrame({'id': ori_test.id, 'trip_duration': np.nan})
    submission.iloc[test_index,-1]=pred
    submission.fillna(submission.trip_duration.mean(),inplace=True)
    submission.to_csv("%s.csv"%p,index=False)

# 走一下流程
+ 前面baseline都好了
+ 只需要修改features ，构建特征和清洗数据即可

In [102]:
%%time 
_=train
x,y=_[features],_.trip_duration
pred_train3,pred_test3=get_oof_XGBoost(x,y,test[features])

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [11:28:18] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-rmse:0.77107	valid_data-rmse:0.77078
[6]	train-rmse:0.63040	valid_data-rmse:0.63264


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [11:28:47] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-rmse:0.77113	valid_data-rmse:0.77055
[6]	train-rmse:0.63042	valid_data-rmse:0.63258


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [11:29:15] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-rmse:0.77046	valid_data-rmse:0.77323
[6]	train-rmse:0.62989	valid_data-rmse:0.63484


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [11:29:43] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-rmse:0.77089	valid_data-rmse:0.77158
[6]	train-rmse:0.63033	valid_data-rmse:0.63335


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [11:30:15] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-rmse:0.77123	valid_data-rmse:0.77027
[6]	train-rmse:0.63057	valid_data-rmse:0.63241
CPU times: user 2min 17s, sys: 7.51 s, total: 2min 24s
Wall time: 2min 26s


In [105]:
makeRes(np.exp(pred_test3.reshape(-1)),'ligtGBM_1')